In [62]:
import six

import pandas as pd
import gffutils
import pybedtools
import re
import numpy as np

v19db_filename = '/projects/ps-yeolab/genomes/hg19/gencode/v19/gencode.v19.annotation.gtf.db'
v19db = gffutils.FeatureDB(v19db_filename)

folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'
outrigger_folder = '{}/outrigger_index'.format(folder)


csv_folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper'
# folder2 = '/projects/ps-yeolab2/obotvinnik/singlecell_pnm'

splicing_feature_data = pd.read_csv('{}/splicing_feature_data.csv'.format(csv_folder), index_col=0)
splicing_feature_data.head()

,criteria,criteria_additional,criteria_full,ensembl_id,exon1,exon1_length,exon2,exon2_length,exon3,exon3_length,...,junction12,junction13,junction23,junction24,junction34,strand,transcript_id,exon2_divisible_by_3,one_ensembl_id,splice_type
event_id,,,,,,,,,,,,,,,,,,,,,
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,99,exon:chr10:100189330-100189399:-,70,...,chr10:100189647-100190327:-,chr10:100189400-100190327:-,chr10:100189400-100189547:-,NaN,NaN,-,"ENST00000361490.4,ENST00000414009.1,ENST000003...",True,ENSG00000107521,SE
exon:chr10:100193697-100193848:-@exon:chr10:100190888-100191048:-@exon:chr10:100190328-100190427:-,only one,NaN,only one,ENSG00000107521,exon:chr10:100193697-100193848:-,152,exon:chr10:100190888-100191048:-,161,exon:chr10:100190328-100190427:-,100,...,chr10:100191049-100193696:-,chr10:100190428-100193696:-,chr10:100190428-100190887:-,NaN,NaN,-,"ENST00000361490.4,ENST00000414009.1,ENST000003...",False,ENSG00000107521,SE
exon:chr10:100195392-100195529:-@exon:chr10:100195029-100195171:-@exon:chr10:100193697-100193848:-,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100195392-100195529:-,138,exon:chr10:100195029-100195171:-,143,exon:chr10:100193697-100193848:-,152,...,chr10:100195172-100195391:-,chr10:100193849-100195391:-,chr10:100193849-100195028:-,NaN,NaN,-,"ENST00000361490.4,ENST00000325103.6,ENST000004...",False,ENSG00000107521,SE
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,only one,NaN,only one,ENSG00000120053,exon:chr10:101165513-101165617:-,105,exon:chr10:101163481-101163631:-,151,exon:chr10:101163226-101163391:-,166,...,chr10:101163632-101165512:-,chr10:101163392-101165512:-,chr10:101163392-101163480:-,NaN,NaN,-,"ENST00000543866.1,ENST00000370508.5",False,ENSG00000120053,SE
exon:chr10:101419263-101419345:+@exon:chr10:101419619-101419721:+@exon:chr10:101421203-101421385:+,only one,NaN,only one,ENSG00000198018,exon:chr10:101419263-101419345:+,83,exon:chr10:101419619-101419721:+,103,exon:chr10:101421203-101421385:+,183,...,chr10:101419346-101419618:+,chr10:101419346-101421202:+,chr10:101419722-101421202:+,NaN,NaN,+,ENST00000370489.4,False,ENSG00000198018,SE


Get only psi scores for events found to actually be alternative in our data (see [0.2.04_Filter_out_non-alternative_splicing_events](0.2.04_Filter_out_non-alternative_splicing_events.ipynb))

In [70]:
ll $outrigger_folder

total 49370
drwxr-xr-x 2 obotvinnik        4 Jan 24 11:03 events/
-rw-r--r-- 1 obotvinnik  2891321 Jan 29 20:00 mxe_psi.csv
-rw-r--r-- 1 obotvinnik 30323119 Jan 30 07:03 psi.csv
-rw-r--r-- 1 obotvinnik 16917226 Jan 30 07:03 se_psi.csv


In [72]:
! wc -l $outrigger_folder/*

wc: /projects/ps-yeolab/obotvinnik/singlecell_pnms/outrigger_index/events: Is a directory
       0 /projects/ps-yeolab/obotvinnik/singlecell_pnms/outrigger_index/events
     240 /projects/ps-yeolab/obotvinnik/singlecell_pnms/outrigger_index/mxe_psi.csv
   43904 /projects/ps-yeolab/obotvinnik/singlecell_pnms/outrigger_index/psi.csv
     243 /projects/ps-yeolab/obotvinnik/singlecell_pnms/outrigger_index/se_psi.csv
   44387 total


In [74]:
mxe = pd.read_csv('{}/mxe_psi.csv'.format(outrigger_folder), index_col=0)
se = pd.read_csv('{}/se_psi.csv'.format(outrigger_folder), index_col=0)

In [77]:
psi = pd.concat([mxe, se], axis=1)
six.print_(psi.shape)
psi.head()

(242, 43903)


,exon:chr10:102003455-102003534:-@exon:chr10:101997779-101997988:-@exon:chr10:101996607-101996726:-@exon:chr10:101995424-101995521:-,exon:chr10:102013178-102013296:-@exon:chr10:102010005-102010089:-@exon:chr10:102003455-102003534:-@exon:chr10:101997940-101997988:-,exon:chr10:102020723-102020801:-@exon:chr10:102019975-102020076:-@exon:chr10:102016019-102016233:-@exon:chr10:102013178-102013296:-,exon:chr10:102021736-102021820:-@exon:chr10:102019975-102020076:-@exon:chr10:102016019-102016233:-@exon:chr10:102013178-102013296:-,exon:chr10:102021736-102021820:-@exon:chr10:102020723-102020785:-@exon:chr10:102016019-102016233:-@exon:chr10:102013178-102013296:-,exon:chr10:102021736-102021820:-@exon:chr10:102020723-102020801:-@exon:chr10:102016019-102016233:-@exon:chr10:102013178-102013296:-,exon:chr10:102021736-102021820:-@exon:chr10:102020723-102020801:-@exon:chr10:102019975-102020076:-@exon:chr10:102013178-102013296:-,exon:chr10:102698352-102698493:+@exon:chr10:102703783-102703885:+@exon:chr10:102705087-102705263:+@exon:chr10:102706796-102706893:+,exon:chr10:102746847-102746953:-@exon:chr10:102746444-102746732:-@exon:chr10:102743705-102743831:-@exon:chr10:102743512-102743574:-,exon:chr10:102746847-102746953:-@exon:chr10:102746506-102746732:-@exon:chr10:102743705-102743831:-@exon:chr10:102743512-102743574:-,...,exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+@exon:chrY:2733129-2733286:+,exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+@exon:chrY:2734834-2735309:+,exon:chrY:2713687-2713784:+@exon:chrY:2733129-2733286:+@exon:chrY:2734834-2735309:+,exon:chrY:2722641-2722812:+@exon:chrY:2733129-2733286:+@exon:chrY:2734834-2734903:+,exon:chrY:2803322-2803487:+@exon:chrY:2821950-2822038:+@exon:chrY:2829115-2829687:+,exon:chrY:2803322-2803487:+@exon:chrY:2821950-2822038:+@exon:chrY:2843136-2843285:+,exon:chrY:2803546-2803810:+@exon:chrY:2821950-2822038:+@exon:chrY:2829115-2829327:+,exon:chrY:2821950-2822038:+@exon:chrY:2829115-2829687:+@exon:chrY:2843136-2843285:+,exon:chrY:2843552-2843695:+@exon:chrY:2844711-2844863:+@exon:chrY:2845981-2846121:+,exon:chrY:7193947-7194210:+@exon:chrY:7209156-7209275:+@exon:chrY:7224176-7224271:+
CVN_01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
CVN_02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CVN_03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CVN_04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
CVN_05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
psi.to_csv('{}/psi.csv'.format(outrigger_folder))

In [87]:
splicing = pd.read_csv('{}/psi.csv'.format(outrigger_folder), index_col=0)
six.print_('splicing.shape', splicing.shape)

splicing.shape (242, 43903)


In [88]:
'exon:chr10:102286732-102286831:-@exon:chr10:102286156-102286311:-@exon:chr10:102283497-102283686:-' in splicing

False

In [89]:
'exon:chr10:102286732-102286831:-@exon:chr10:102286156-102286311:-@exon:chr10:102283497-102283686:-' in splicing_feature_data.index

False

In [90]:
bed_folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms/bed'

In [91]:
# junction_exons = pd.read_csv('{}/junction_exon_triples.csv'.format(folder))
# print junction_exons.shape
# junction_exons.head()

## Get exon2 from SE and MXE events

The `splicing` matrix includes both SE and MXE events. The second exon in each of them is "exon2", or the exon which is included when Psi=1.

In [92]:
splicing.head()

,exon:chr10:102003455-102003534:-@exon:chr10:101997779-101997988:-@exon:chr10:101996607-101996726:-@exon:chr10:101995424-101995521:-,exon:chr10:102013178-102013296:-@exon:chr10:102010005-102010089:-@exon:chr10:102003455-102003534:-@exon:chr10:101997940-101997988:-,exon:chr10:102020723-102020801:-@exon:chr10:102019975-102020076:-@exon:chr10:102016019-102016233:-@exon:chr10:102013178-102013296:-,exon:chr10:102021736-102021820:-@exon:chr10:102019975-102020076:-@exon:chr10:102016019-102016233:-@exon:chr10:102013178-102013296:-,exon:chr10:102021736-102021820:-@exon:chr10:102020723-102020785:-@exon:chr10:102016019-102016233:-@exon:chr10:102013178-102013296:-,exon:chr10:102021736-102021820:-@exon:chr10:102020723-102020801:-@exon:chr10:102016019-102016233:-@exon:chr10:102013178-102013296:-,exon:chr10:102021736-102021820:-@exon:chr10:102020723-102020801:-@exon:chr10:102019975-102020076:-@exon:chr10:102013178-102013296:-,exon:chr10:102698352-102698493:+@exon:chr10:102703783-102703885:+@exon:chr10:102705087-102705263:+@exon:chr10:102706796-102706893:+,exon:chr10:102746847-102746953:-@exon:chr10:102746444-102746732:-@exon:chr10:102743705-102743831:-@exon:chr10:102743512-102743574:-,exon:chr10:102746847-102746953:-@exon:chr10:102746506-102746732:-@exon:chr10:102743705-102743831:-@exon:chr10:102743512-102743574:-,...,exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+@exon:chrY:2733129-2733286:+,exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+@exon:chrY:2734834-2735309:+,exon:chrY:2713687-2713784:+@exon:chrY:2733129-2733286:+@exon:chrY:2734834-2735309:+,exon:chrY:2722641-2722812:+@exon:chrY:2733129-2733286:+@exon:chrY:2734834-2734903:+,exon:chrY:2803322-2803487:+@exon:chrY:2821950-2822038:+@exon:chrY:2829115-2829687:+,exon:chrY:2803322-2803487:+@exon:chrY:2821950-2822038:+@exon:chrY:2843136-2843285:+,exon:chrY:2803546-2803810:+@exon:chrY:2821950-2822038:+@exon:chrY:2829115-2829327:+,exon:chrY:2821950-2822038:+@exon:chrY:2829115-2829687:+@exon:chrY:2843136-2843285:+,exon:chrY:2843552-2843695:+@exon:chrY:2844711-2844863:+@exon:chrY:2845981-2846121:+,exon:chrY:7193947-7194210:+@exon:chrY:7209156-7209275:+@exon:chrY:7224176-7224271:+
CVN_01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
CVN_02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CVN_03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CVN_04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
CVN_05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
exon2s = pd.Series(splicing.columns.map(lambda x: x.split('@')[1]))
len(exon2s)
exon2s[:10]

0    exon:chr10:101997779-101997988:-
1    exon:chr10:102010005-102010089:-
2    exon:chr10:102019975-102020076:-
3    exon:chr10:102019975-102020076:-
4    exon:chr10:102020723-102020785:-
5    exon:chr10:102020723-102020801:-
6    exon:chr10:102020723-102020801:-
7    exon:chr10:102703783-102703885:+
8    exon:chr10:102746444-102746732:-
9    exon:chr10:102746506-102746732:-
dtype: object

### Write Exon2 from SE and MXE to bed file

In [94]:
from outrigger.region import Region

exon2_regions = exon2s.map(Region)

exon2_bed = pd.DataFrame.from_records(exon2_regions.map(
    lambda x: pd.Series(dict(chrom=x.chrom, start=x._start, stop=x._stop, strand=x.strand, score=1000))))

exon2_bed['name'] = splicing.columns
exon2_bed = exon2_bed.reindex(columns=['chrom', 'start', 'stop', 'name', 'score', 'strand'])

print exon2_bed.shape
exon2_bed.head()

(43903, 6)


,chrom,start,stop,name,score,strand
0,chr10,101997779,101997988,exon:chr10:102003455-102003534:-@exon:chr10:10...,1000,-
1,chr10,102010005,102010089,exon:chr10:102013178-102013296:-@exon:chr10:10...,1000,-
2,chr10,102019975,102020076,exon:chr10:102020723-102020801:-@exon:chr10:10...,1000,-
3,chr10,102019975,102020076,exon:chr10:102021736-102021820:-@exon:chr10:10...,1000,-
4,chr10,102020723,102020785,exon:chr10:102021736-102021820:-@exon:chr10:10...,1000,-


In [95]:
event = 'exon:chr10:1043148-1043248:+@exon:chr10:1044943-1045035:+@exon:chr10:1046617-1046808:+'

In [96]:
event in exon2_bed.name

False

In [97]:
exon2_bed.to_csv('{}/exons.bed'.format(alternative_feature_folder), index=False, header=False, sep='\t')

## Get constitutive exons

### Use exons that are psi==1 in all celltypes

In [98]:
constutitively1 = pd.read_csv('{}/psi_constitutively1.csv'.format(csv_folder), index_col=0)
six.print_('constutitively1.shape', constutitively1.shape)
constutitively1.head()

constutitively1.shape (242, 9161)


,exon:chr10:101979027-101979116:-@exon:chr10:101978765-101978889:-@exon:chr10:101978475-101978582:-,exon:chr10:102021736-102021820:-@exon:chr10:102020723-102020785:-@exon:chr10:102019975-102020076:-,exon:chr10:102021736-102021820:-@exon:chr10:102020723-102020801:-@exon:chr10:102019975-102020076:-,exon:chr10:102046362-102046430:-@exon:chr10:102045854-102045970:-@exon:chr10:102040691-102040810:-,exon:chr10:102046362-102046469:-@exon:chr10:102045854-102046089:-@exon:chr10:102040691-102040810:-,exon:chr10:103344359-103344676:-@exon:chr10:103343265-103343438:-@exon:chr10:103342520-103342648:-,exon:chr10:103347003-103347883:-@exon:chr10:103343265-103343438:-@exon:chr10:103342520-103342648:-,exon:chr10:103345619-103345913:-@exon:chr10:103345073-103345235:-@exon:chr10:103344368-103344676:-,exon:chr10:103354459-103354495:+@exon:chr10:103360495-103360619:+@exon:chr10:103360960-103361093:+,exon:chr10:103432672-103432804:-@exon:chr10:103427643-103427737:-@exon:chr10:103384502-103384567:-,...,exon:chrX:80377034-80377171:-@exon:chrX:80375287-80375316:-@exon:chrX:80374229-80374279:-@exon:chrX:80373931-80373984:-,exon:chrX:85302488-85302543:-@exon:chrX:85282495-85282561:-@exon:chrX:85236741-85236813:-@exon:chrX:85233771-85233895:-,exon:chrX:99888402-99888536:-@exon:chrX:99887482-99887565:-@exon:chrX:99885756-99885863:-@exon:chrX:99883667-99884983:-,exon:chrY:14774284-14774637:+@exon:chrY:14776571-14776617:+@exon:chrY:14798442-14798535:+@exon:chrY:14821321-14821476:+,exon:chrY:14798442-14798535:+@exon:chrY:14802255-14802370:+@exon:chrY:14820567-14820626:+@exon:chrY:14821321-14821476:+,exon:chrY:15024920-15024974:+@exon:chrY:15025630-15025765:+@exon:chrY:15026476-15026561:+@exon:chrY:15026979-15027139:+,exon:chrY:15024875-15024974:+@exon:chrY:15025630-15025765:+@exon:chrY:15026796-15026894:+@exon:chrY:15026979-15027139:+,exon:chrY:2712118-2712298:+@exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+@exon:chrY:2734834-2734903:+,exon:chrY:2803112-2803487:+@exon:chrY:2821950-2822038:+@exon:chrY:2829115-2829687:+@exon:chrY:2843136-2843285:+,exon:chrY:2803518-2803810:+@exon:chrY:2821950-2822038:+@exon:chrY:2829115-2829687:+@exon:chrY:2843136-2843285:+
CVN_01,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CVN_02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CVN_03,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CVN_04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CVN_05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
# This metadata only has iPSC, NPC, and MN (no sMN)

metadata = pd.read_csv('{}/metadata.csv'.format(csv_folder), index_col=0)
metadata.head()

,single,pooled,phenotype,neuron,progenitor,stressed,craig_venter,differentiation_batch,split
CVN_01,True,False,NPC,False,True,False,True,NaN,NaN
CVN_02,True,False,NPC,False,True,False,True,NaN,NaN
CVN_03,True,False,NPC,False,True,False,True,NaN,NaN
CVN_04,True,False,NPC,False,True,False,True,NaN,NaN
CVN_05,True,False,NPC,False,True,False,True,NaN,NaN


In [100]:
counts = constutitively1.groupby(metadata.phenotype).count()
counts

,exon:chr10:101979027-101979116:-@exon:chr10:101978765-101978889:-@exon:chr10:101978475-101978582:-,exon:chr10:102021736-102021820:-@exon:chr10:102020723-102020785:-@exon:chr10:102019975-102020076:-,exon:chr10:102021736-102021820:-@exon:chr10:102020723-102020801:-@exon:chr10:102019975-102020076:-,exon:chr10:102046362-102046430:-@exon:chr10:102045854-102045970:-@exon:chr10:102040691-102040810:-,exon:chr10:102046362-102046469:-@exon:chr10:102045854-102046089:-@exon:chr10:102040691-102040810:-,exon:chr10:103344359-103344676:-@exon:chr10:103343265-103343438:-@exon:chr10:103342520-103342648:-,exon:chr10:103347003-103347883:-@exon:chr10:103343265-103343438:-@exon:chr10:103342520-103342648:-,exon:chr10:103345619-103345913:-@exon:chr10:103345073-103345235:-@exon:chr10:103344368-103344676:-,exon:chr10:103354459-103354495:+@exon:chr10:103360495-103360619:+@exon:chr10:103360960-103361093:+,exon:chr10:103432672-103432804:-@exon:chr10:103427643-103427737:-@exon:chr10:103384502-103384567:-,...,exon:chrX:80377034-80377171:-@exon:chrX:80375287-80375316:-@exon:chrX:80374229-80374279:-@exon:chrX:80373931-80373984:-,exon:chrX:85302488-85302543:-@exon:chrX:85282495-85282561:-@exon:chrX:85236741-85236813:-@exon:chrX:85233771-85233895:-,exon:chrX:99888402-99888536:-@exon:chrX:99887482-99887565:-@exon:chrX:99885756-99885863:-@exon:chrX:99883667-99884983:-,exon:chrY:14774284-14774637:+@exon:chrY:14776571-14776617:+@exon:chrY:14798442-14798535:+@exon:chrY:14821321-14821476:+,exon:chrY:14798442-14798535:+@exon:chrY:14802255-14802370:+@exon:chrY:14820567-14820626:+@exon:chrY:14821321-14821476:+,exon:chrY:15024920-15024974:+@exon:chrY:15025630-15025765:+@exon:chrY:15026476-15026561:+@exon:chrY:15026979-15027139:+,exon:chrY:15024875-15024974:+@exon:chrY:15025630-15025765:+@exon:chrY:15026796-15026894:+@exon:chrY:15026979-15027139:+,exon:chrY:2712118-2712298:+@exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+@exon:chrY:2734834-2734903:+,exon:chrY:2803112-2803487:+@exon:chrY:2821950-2822038:+@exon:chrY:2829115-2829687:+@exon:chrY:2843136-2843285:+,exon:chrY:2803518-2803810:+@exon:chrY:2821950-2822038:+@exon:chrY:2829115-2829687:+@exon:chrY:2843136-2843285:+
phenotype,,,,,,,,,,,,,,,,,,,,,
MN,0,2,10,35,5,3,1,3,29,0,...,0,0,0,3,0,1,1,0,0,0
NPC,1,1,12,41,3,1,0,0,42,1,...,1,1,1,0,1,0,0,0,0,0
iPSC,4,11,27,44,0,1,1,0,33,1,...,0,0,5,3,0,0,0,0,2,1


In [101]:
at_least_20 = (counts >= 20).all()
constitutive_events = at_least_20.index[at_least_20]

In [102]:
constitutive_exon2s = pd.Series(constitutive_events.map(lambda x: x.split('@')[1]))
constitutive_exon2s.head()

0    exon:chr10:102045854-102045970:-
1    exon:chr10:103360495-103360619:+
2    exon:chr10:103916776-103916831:+
3        exon:chr10:1043148-1043248:+
4    exon:chr10:104445573-104445758:-
dtype: object

### Internal: have both downstream and upstream junctions

In [103]:
junction_exons = pd.read_csv('{}/junction_exon_triples.csv'.format(csv_folder))
print junction_exons.shape
junction_exons.head()

(844056, 3)


,exon,direction,junction
0,exon:chr1:29321-29370:-,upstream,chr1:18367-29320:-
1,exon:chr1:29321-29370:-,upstream,chr1:24892-29320:-
2,exon:chr1:29321-29370:-,upstream,chr1:27186-29320:-
3,exon:chr1:29321-29370:-,upstream,chr1:18380-29320:-
4,exon:chr1:29321-29370:-,upstream,chr1:18370-29320:-


In [104]:
exon_direction = junction_exons.groupby(['exon', 'direction']).size()
exon_direction.head()
# constitutive = constitutive[constitutive == 2]

exon                              direction 
exon:chr10:100003848-100004106:+  downstream    1
exon:chr10:100003848-100004321:+  downstream    1
exon:chr10:100003848-100004651:+  downstream    1
exon:chr10:100003848-100004654:+  downstream    1
exon:chr10:100007447-100008748:-  downstream    1
dtype: int64

Group on the number of "direction" items they have. Must have two to be internal ("upstream" and "downstream")

In [105]:
internal_exons = exon_direction.groupby(level=0).size()
internal_exons = internal_exons[internal_exons == 2]
internal_exons.head()

exon
exon:chr10:100010822-100010933:-    2
exon:chr10:100144704-100144824:-    2
exon:chr10:100146958-100147064:-    2
exon:chr10:100150355-100150511:-    2
exon:chr10:100155148-100155209:-    2
dtype: int64

In [106]:
exon_juction_direction_internal = junction_exons.loc[junction_exons.exon.isin(internal_exons.index)]
print exon_juction_direction_internal.shape
exon_juction_direction_internal.head()

(448078, 3)


,exon,direction,junction
5,exon:chr1:24738-24891:-,downstream,chr1:24892-29320:-
6,exon:chr1:24738-24891:-,downstream,chr1:24892-29533:-
7,exon:chr1:24738-24891:-,upstream,chr1:18062-24737:-
8,exon:chr1:24738-24891:-,upstream,chr1:18380-24737:-
9,exon:chr1:24738-24891:-,upstream,chr1:18555-24737:-


### Remove all exons that appear as exon2 in any alternative annotations, or exon3 for MXE

In [107]:
splicing_types = pd.Series(['MXE' if len(x.split('@')) == 4 else 'SE' for x in splicing.columns], index=splicing.columns)
six.print_(splicing_types.shape)
splicing_types.head()

(43903,)


exon:chr10:102003455-102003534:-@exon:chr10:101997779-101997988:-@exon:chr10:101996607-101996726:-@exon:chr10:101995424-101995521:-    MXE
exon:chr10:102013178-102013296:-@exon:chr10:102010005-102010089:-@exon:chr10:102003455-102003534:-@exon:chr10:101997940-101997988:-    MXE
exon:chr10:102020723-102020801:-@exon:chr10:102019975-102020076:-@exon:chr10:102016019-102016233:-@exon:chr10:102013178-102013296:-    MXE
exon:chr10:102021736-102021820:-@exon:chr10:102019975-102020076:-@exon:chr10:102016019-102016233:-@exon:chr10:102013178-102013296:-    MXE
exon:chr10:102021736-102021820:-@exon:chr10:102020723-102020785:-@exon:chr10:102016019-102016233:-@exon:chr10:102013178-102013296:-    MXE
dtype: object

In [108]:
mxe_exon3s = pd.Series([x.split('@')[2] for x in splicing.columns[splicing_types == 'MXE']])
mxe_exon3s

0       exon:chr10:101996607-101996726:-
1       exon:chr10:102003455-102003534:-
2       exon:chr10:102016019-102016233:-
3       exon:chr10:102016019-102016233:-
4       exon:chr10:102016019-102016233:-
5       exon:chr10:102016019-102016233:-
6       exon:chr10:102019975-102020076:-
7       exon:chr10:102705087-102705263:+
8       exon:chr10:102743705-102743831:-
9       exon:chr10:102743705-102743831:-
10      exon:chr10:102743791-102743831:-
11      exon:chr10:102743791-102743831:-
12      exon:chr10:102746444-102746732:-
13      exon:chr10:102746506-102746732:-
14      exon:chr10:103213258-103213313:+
15      exon:chr10:103221738-103221815:+
16      exon:chr10:103342520-103342648:-
17      exon:chr10:103343265-103343438:-
18      exon:chr10:103346879-103346957:-
19      exon:chr10:103360495-103360619:+
20      exon:chr10:103368592-103368694:+
21      exon:chr10:103368624-103368694:+
22      exon:chr10:103368624-103368801:+
23          exon:chr10:1041869-1041972:+
24      exon:chr

In [109]:
alt_exons = pd.concat([exon2s, mxe_exon3s], ignore_index=True)
print alt_exons.shape
alt_exons = alt_exons.drop_duplicates()
print alt_exons.shape

in_alt_exons = exon_juction_direction_internal.exon.isin(alt_exons.values)

constitutive = exon_juction_direction_internal.loc[(~in_alt_exons)]
print constitutive.shape
constitutive.head()

(51155,)
(29394,)
(338179, 3)


,exon,direction,junction
5,exon:chr1:24738-24891:-,downstream,chr1:24892-29320:-
6,exon:chr1:24738-24891:-,downstream,chr1:24892-29533:-
7,exon:chr1:24738-24891:-,upstream,chr1:18062-24737:-
8,exon:chr1:24738-24891:-,upstream,chr1:18380-24737:-
9,exon:chr1:24738-24891:-,upstream,chr1:18555-24737:-


In [110]:
from outrigger.region import Region
constitutive_regions = pd.Series(constitutive.exon.unique()).map(Region)
print constitutive_regions.shape
constitutive_regions.head()

(120552,)


0    exon:chr1:24738-24891:-
1    exon:chr1:18268-18379:-
2    exon:chr1:17915-18061:-
3    exon:chr1:16854-17055:-
4    exon:chr1:16607-16765:-
dtype: object

In [111]:
constitutive_junctions = constitutive.junction.unique()
len(constitutive_junctions)

222439

### Filter constitutive junctions for usage

Check that the junctions for these exons are seen in 20+ single cells per celltype

In [112]:
! ls $csv_folder

conservation
constitutive_downstream400_pyrimidine_content.csv
constitutive_exons.bed
constitutive_feature_data.csv
constitutive_upstream400_pyrimidine_content.csv
exon2_downstream400_pyrimidine_content.csv
exon2_upstream400_pyrimidine_content.csv
expression.csv
expression_stressed.csv
junction_exon_triples.csv
mapping_stats.csv
mapping_stats_stressed.csv
metadata.csv
metadata_stressed.csv
mutually_exclusive_exon_consolidated_events.csv
mutually_exclusive_exons_psi.csv
mxe_events_with_stressed.csv
nucleotide_content
outrigger_singlecell.sh
outrigger_singlecell.sh.err
outrigger_singlecell.sh.out
outrigger_singlecell.sh~
psi.csv
psi3.csv
psi3_stressed.csv
psi5.csv
psi5_psi3_combined.csv
psi5_psi3_combined.hdf
psi5_psi3_combined_metadata.csv
psi5_psi3_combined_metadata.hdf
psi5_stressed.csv
psi_constitutively0.csv
psi_constitutively1.csv
psi_unfiltered.csv
se_events_with_stressed.csv
sj_metadata.csv
sj_raw.csv
skipped_exon_consolidated_events.csv
skipped_exon_psi_from_junctions.csv
splice

In [113]:
sj = pd.read_csv('{}/sj_raw.csv'.format(csv_folder))

sj.head()

,chrom,intron_start,intron_stop,strand,intron_motif,annotated,unique_junction_reads,multimap_junction_reads,max_overhang,intron_location,sample_id
0,chr1,14830,14969,-,GT/AG,True,0,1,35,chr1:14830-14969:-,CVN_09
1,chr1,15039,15795,-,GT/AG,True,0,1,49,chr1:15039-15795:-,CVN_09
2,chr1,228777,237053,-,GT/AG,True,2,12,48,chr1:228777-237053:-,CVN_09
3,chr1,320654,324287,+,GT/AG,True,0,1,28,chr1:320654-324287:+,CVN_09
4,chr1,320939,321031,+,GT/AG,True,0,3,39,chr1:320939-321031:+,CVN_09


### Get just junctions associated with constitutive exons

In [114]:
sj_constitutive = sj.loc[sj['intron_location'].isin(constitutive_junctions)]
print sj_constitutive.shape
sj_constitutive.head()

(5054625, 11)


,chrom,intron_start,intron_stop,strand,intron_motif,annotated,unique_junction_reads,multimap_junction_reads,max_overhang,intron_location,sample_id
1,chr1,15039,15795,-,GT/AG,True,0,1,49,chr1:15039-15795:-,CVN_09
2,chr1,228777,237053,-,GT/AG,True,2,12,48,chr1:228777-237053:-,CVN_09
8,chr1,1309283,1309379,-,GT/AG,True,37,0,49,chr1:1309283-1309379:-,CVN_09
9,chr1,1309826,1310084,-,GT/AG,True,43,0,49,chr1:1309826-1310084:-,CVN_09
11,chr1,1337637,1341188,-,GT/AG,True,409,0,50,chr1:1337637-1341188:-,CVN_09


In [115]:
# This metadata only has iPSC, NPC, and MN (no sMN)

metadata = pd.read_csv('{}/metadata.csv'.format(csv_folder), index_col=0)
metadata.head()

,single,pooled,phenotype,neuron,progenitor,stressed,craig_venter,differentiation_batch,split
CVN_01,True,False,NPC,False,True,False,True,NaN,NaN
CVN_02,True,False,NPC,False,True,False,True,NaN,NaN
CVN_03,True,False,NPC,False,True,False,True,NaN,NaN
CVN_04,True,False,NPC,False,True,False,True,NaN,NaN
CVN_05,True,False,NPC,False,True,False,True,NaN,NaN


In [116]:
metadata.groupby('phenotype').size()

phenotype
MN      73
NPC     76
iPSC    65
dtype: int64

In [117]:
ls $csv_folder

conservation/
constitutive_downstream400_pyrimidine_content.csv
constitutive_exons.bed
constitutive_feature_data.csv
constitutive_upstream400_pyrimidine_content.csv
exon2_downstream400_pyrimidine_content.csv
exon2_upstream400_pyrimidine_content.csv
expression.csv
expression_stressed.csv
junction_exon_triples.csv
mapping_stats.csv
mapping_stats_stressed.csv
metadata.csv
metadata_stressed.csv
mutually_exclusive_exon_consolidated_events.csv
mutually_exclusive_exons_psi.csv
mxe_events_with_stressed.csv
nucleotide_content/
outrigger_singlecell.sh
outrigger_singlecell.sh.err
outrigger_singlecell.sh.out
outrigger_singlecell.sh~
psi.csv
psi3.csv
psi3_stressed.csv
psi5.csv
psi5_psi3_combined.csv
psi5_psi3_combined.hdf
psi5_psi3_combined_metadata.csv
psi5_psi3_combined_metadata.hdf
psi5_stressed.csv
psi_constitutively0.csv
psi_constitutively1.csv
psi_unfiltered.csv
se_events_with_stressed.csv
sj_metadata.csv
sj_raw.csv
skipped_exon_consolidated_events.csv
skipped_exon_psi_from_junctions.csv
spli

Get only single cells for calculating junction usage (no pooled)

In [118]:
phenotype = metadata.phenotype[metadata.single]

### Call an exon constitutive if its junction start/stop is detected in at least 20 cells per celltype

In [119]:
sj_constitutive = sj_constitutive.set_index('sample_id')
sj_constitutive = sj_constitutive.join(phenotype)
sj_constitutive.head()

,chrom,intron_start,intron_stop,strand,intron_motif,annotated,unique_junction_reads,multimap_junction_reads,max_overhang,intron_location,phenotype
CVN_01,chr1,1328184,1328775,-,GT/AG,True,20,0,50,chr1:1328184-1328775:-,NPC
CVN_01,chr1,1337637,1341188,-,GT/AG,True,362,0,50,chr1:1337637-1341188:-,NPC
CVN_01,chr1,1341267,1342288,-,GT/AG,True,312,0,50,chr1:1341267-1342288:-,NPC
CVN_01,chr1,1342400,1342510,-,GT/AG,True,246,0,48,chr1:1342400-1342510:-,NPC
CVN_01,chr1,1572367,1572442,-,GT/AG,True,0,2,8,chr1:1572367-1572442:-,NPC


### Use at least 10 unique junction reads

In [120]:
print sj_constitutive.shape
sj_constitutive = sj_constitutive.loc[sj_constitutive.unique_junction_reads >= 10]
print sj_constitutive.shape


(5054625, 11)
(3591047, 11)


In [121]:
%%time

min_samples = 20

chrom = 'chrom'
start = 'intron_start'
stop = 'intron_stop'
strand = 'strand'


constitutive_starts = sj_constitutive.groupby('phenotype').apply(
    lambda x: x.groupby([chrom, start, strand]).filter( lambda y: len(y.index.unique()) > min_samples ))
    

CPU times: user 2min 8s, sys: 1 s, total: 2min 9s
Wall time: 2min 9s


In [122]:
%%time 

constitutive_stops = sj_constitutive.groupby('phenotype').apply(
    lambda x: x.groupby([chrom, stop, strand]).filter(lambda y: len(y.index.unique()) > min_samples ))


CPU times: user 2min 8s, sys: 905 ms, total: 2min 9s
Wall time: 2min 9s


### Filter for constitutive stops and starts that are in ALL phenotypes

In [123]:
constitutive_stops.groupby('phenotype').size()

phenotype
MN      268214
NPC     468667
iPSC    869063
dtype: int64

In [124]:
%%time 
constitutive_stops_all_phenotypes = constitutive_stops.groupby('intron_location').filter(
    lambda x: len(x.groupby('phenotype')) == 3)

CPU times: user 25.9 s, sys: 42 ms, total: 25.9 s
Wall time: 26 s


In [125]:
%%time
constitutive_starts_all_phenotypes = constitutive_starts.groupby('intron_location').filter(
    lambda x: len(x.groupby('phenotype')) == 3)

CPU times: user 26.4 s, sys: 106 ms, total: 26.5 s
Wall time: 26.6 s


In [126]:
constitutive_stops_all_phenotypes.head()

chrom  intron_start  intron_stop strand intron_motif  \
phenotype                                                              
MN        M1_01  chr1       1337637      1341188      -        GT/AG   
          M1_01  chr1       1341267      1342288      -        GT/AG   
          M1_01  chr1       1342400      1342510      -        GT/AG   
          M1_01  chr1       6253115      6257711      -        GT/AG   
          M1_01  chr1       6257817      6259621      -        GT/AG   

                annotated  unique_junction_reads  multimap_junction_reads  \
phenotype                                                                   
MN        M1_01      True                    588                        0   
          M1_01      True                    341                        0   
          M1_01      True                    227                        0   
          M1_01      True                    131                        9   
          M1_01      True                    114                        0   

                 max_overhang         intron_location phenotype  
phenotype                                                        
MN        M1_01            45  chr1:1337637-1341188:-        MN  
          M1_01            46  chr1:1341267-1342288:-        MN  
          M1_01            46  chr1:1342400-1342510:-        MN  
          M1_01            45  chr1:6253115-6257711:-        MN  
          M1_01            35  chr1:6257817-6259621:-        MN

In [127]:
constitutive_upstream_junctions = constitutive_stops_all_phenotypes.intron_location.unique()
constitutive_downstream_junctions = constitutive_starts_all_phenotypes.intron_location.unique()

In [128]:
grouped = constitutive.groupby('direction')
upstream = grouped.get_group('upstream')
downstream = grouped.get_group('downstream')

Get exons which have both junctions in the list of constitutive

In [129]:
%%time
constitutive_both_junctions = constitutive.groupby('exon').filter(
    lambda x: bool(x['junction'].isin(constitutive_downstream_junctions).sum() 
                   & x['junction'].isin(constitutive_upstream_junctions).sum()))

CPU times: user 4min 33s, sys: 609 ms, total: 4min 34s
Wall time: 4min 34s


In [130]:
constitutive_both_junctions.head()

,exon,direction,junction
1525,exon:chr1:1341189-1341266:-,downstream,chr1:1341267-1342288:-
1526,exon:chr1:1341189-1341266:-,downstream,chr1:1341267-1341605:-
1527,exon:chr1:1341189-1341266:-,downstream,chr1:1341267-1342215:-
1528,exon:chr1:1341189-1341266:-,downstream,chr1:1341267-1342367:-
1529,exon:chr1:1341189-1341266:-,downstream,chr1:1341267-1342510:-


In [131]:
splicing_feature_folder = '{}/splicing_feature_data'.format(csv_folder)
alternative_feature_folder = '{}/alternative'.format(splicing_feature_folder)
constitutive_feature_folder = '{}/constitutive'.format(splicing_feature_folder)


In [132]:
constitutive_both_junctions.to_csv('{}/constitutive_junctions.csv'.format(constitutive_feature_folder), index=False)

In [133]:
constitutive_both_junctions.shape

(21195, 3)

In [134]:
len(constitutive_both_junctions.exon.unique())

5293

#### Make "constitutive_exons"

In [135]:
constitutive_exons = constitutive_both_junctions.exon
constitutive_exons[:10]

1525    exon:chr1:1341189-1341266:-
1526    exon:chr1:1341189-1341266:-
1527    exon:chr1:1341189-1341266:-
1528    exon:chr1:1341189-1341266:-
1529    exon:chr1:1341189-1341266:-
1530    exon:chr1:1341189-1341266:-
1531    exon:chr1:1341189-1341266:-
1532    exon:chr1:1341189-1341266:-
1533    exon:chr1:1341189-1341266:-
1534    exon:chr1:1341189-1341266:-
Name: exon, dtype: object

In [136]:
constitutive_exons.shape

(21195,)

In [137]:
constitutive_exons_no_duplicates = pd.Series(constitutive_exons)
six.print_(constitutive_exons_no_duplicates.shape)
constitutive_exons_no_duplicates = constitutive_exons_no_duplicates.drop_duplicates()
six.print_(constitutive_exons_no_duplicates.shape)

(21195,)
(5293,)


### Add everything from psi_constitutively1

In [138]:
all_constitutive_exons = pd.concat([constitutive_exons_no_duplicates, constitutive_exon2s])
six.print_(all_constitutive_exons.shape)
all_constitutive_exons = all_constitutive_exons.drop_duplicates()
six.print_(all_constitutive_exons.shape)

(5897,)
(5893,)


### Make BED file of constitutive

In [139]:
constitutive_regions = all_constitutive_exons.map(Region)

constitutive_bed_table = pd.DataFrame(
    map(lambda x: [x.chrom, x._start, x._stop, x.name, 1000, x.strand], constitutive_regions))
constitutive_bed_table[3] = map(str, constitutive_regions)
constitutive_bed_table = constitutive_bed_table.sort_values(by=3)

constitutive_bed_table = constitutive_bed_table.drop_duplicates()
print constitutive_bed_table.shape
constitutive_bed_table.head()

(5893, 6)


,0,1,2,3,4,5
2669,chr10,102039881,102039985,exon:chr10:102039881-102039985:-,1000,-
2668,chr10,102040691,102040810,exon:chr10:102040691-102040810:-,1000,-
5293,chr10,102045854,102045970,exon:chr10:102045854-102045970:-,1000,-
2671,chr10,102286732,102286831,exon:chr10:102286732-102286831:-,1000,-
2672,chr10,102289123,102289263,exon:chr10:102289123-102289263:-,1000,-


In [140]:
constitutive_bed_table.to_csv('{}/exons.bed'.format(bed_folder), sep='\t', index=False, header=False)

In [144]:
constitutive_bed_table.to_csv('{}/exons.bed'.format(constitutive_feature_folder), sep='\t', index=False, header=False)

In [145]:
ls $constitutive_feature_folder

constitutive_exons_phastcons_placental_mammal.bed
constitutive_exons_phastcons_placental_mammal.txt
constitutive_junctions.csv
downstream400_placental_mammal_conservation.csv
exon2_merkin2012_ancient.csv
exon_length.csv
exons.bed
exons.fasta
exons_merkin2012_ancient_alt_exons.bed
exons_rnahybrid_mirbase_results.txt
first_last_internal.csv
gc_content.csv
genes.csv
genes_expanded.csv
intron_length.csv
maxentscan.csv
nucleotide_content/
phastcons_placental_mammal.txt@
repetitive_element_overlap.csv
upstream400_placental_mammal_conservation.csv
